In [1]:
import numpy as np 

In [2]:
diffs = {'synthetic': {}, 'processed': {}}
for data in ['processed', 'synthetic']:
    for lang in ['en', 'ga', 'de']:
        print(data, lang)
        denoised_scores = [float(i) for i in open(f'scores/{data}/denoised_scores_{lang}').readlines()]
        syn_scores = [float(i) for i in open(f'scores/{data}/syn_scores_{lang}').readlines()]
        if(data == 'processed'):
            syn_scores = syn_scores[:-1]
        diffs[data][lang] = np.array(denoised_scores) - np.array(syn_scores)

processed en


FileNotFoundError: [Errno 2] No such file or directory: 'scores/processed/denoised_scores_en'

In [12]:
for data in diffs:
    for lang in diffs[data]:
        num_positive = len([i for i in diffs[data][lang] if i > 0])
        print(data, lang, np.mean(diffs[data][lang]), np.std(diffs[data][lang]), num_positive)

synthetic en -1.0641810917480914 0.7702603745846928 6250
synthetic ga -0.9350833649630985 0.6833910482771184 123
synthetic de -1.0704608548300845 0.7203212966942848 2242
processed en 1.657660816279002 0.621742158611325 34349
processed ga 1.8709259184191953 0.7756332226565065 35421
processed de 1.7828836510059631 0.664539469541866 17942


In [33]:
for lang in ['en', 'ga', 'de']:
    syn_src = open(f'/home2/aditya_hari/gsoc/data/synthetic/{lang}/train_src', 'r').readlines()[:-1]
    syn_tgt = open(f'/home2/aditya_hari/gsoc/data/synthetic/{lang}/train_tgt', 'r').readlines()[:-1]

    trusted_src = open(f'/home2/aditya_hari/gsoc/data/processed/{lang}/train_src', 'r').readlines()
    trusted_tgt = open(f'/home2/aditya_hari/gsoc/data/processed/{lang}/train_tgt', 'r').readlines()
    
    for iter in range(1, 4):
        print(iter, lang)
        srcs = []
        tgts = []

        srcs.extend(trusted_src)
        tgts.extend(trusted_tgt)
        score_diff = diffs['synthetic'][lang]
        triples = list(zip(syn_src, syn_tgt, score_diff))
        triples = sorted(triples, key=lambda x: x[2], reverse=True)
        for src, tgt, score in triples:
            if(score > 0):
                srcs.append(src)
                tgts.append(tgt)

        if(iter < 3):
            non_positives = [i for i in triples if i[2] < 0][:len(triples)//(2*iter)]   
            print(len(non_positives)) 
            # sample 50000 sentences 
            if(len(non_positives) > 50000):
                sample_indices = np.random.choice(len(non_positives), 50000)
            else:
                sample_indices = np.random.choice(len(non_positives), len(non_positives))
            for i in sample_indices:
                srcs.append(non_positives[i][0])
                tgts.append(non_positives[i][1])

        print(len(srcs), len(tgts))
        with open(f'/home2/aditya_hari/gsoc/rdf-to-text/src/denoising/data/iter{iter-1}/{lang}/train_src', 'w') as f:
            f.write(''.join(srcs))
        with open(f'/home2/aditya_hari/gsoc/rdf-to-text/src/denoising/data/iter{iter-1}/{lang}/train_tgt', 'w') as f:
            f.write(''.join(tgts))

1 en
313685
90602 90602
2 en
156842
90602 90602
3 en
40602 40602
1 ga
9010
44559 44559
2 ga
4505
40054 40054
3 ga
35549 35549
1 de
238427
70186 70186
2 de
119213
70186 70186
3 de
20186 20186
